### Problem statement

    - Which store has the farthest distance to its nearest neighboring store?
    - What % of stores are within 1km, 3km and 5km of another store?
    - Which store has the most other stores within 10km?

In [1]:
# load libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# load data
df = pd.read_csv('store_locations.csv')

In [3]:
# sample
df.sample(5)

,Store,lon,lat
40,2733,-1.317660,54.5878
163,2198,-0.936305,51.4404
41,1939,-1.008210,51.6511
208,1920,-1.426600,51.5800
112,233,0.280909,50.7907


In [4]:
# describe
df.describe()

,Store,lon,lat
count,300.000000,300.000000,300.000000
mean,1394.860000,-2.124941,53.008898
std,872.449717,1.952026,1.916977
min,6.000000,-7.879090,49.955600
25%,612.000000,-3.293575,51.494975
50%,1308.000000,-1.971740,52.454400
75%,2120.000000,-0.332130,54.080725
max,2940.000000,1.301920,57.908400


In [5]:
# sample solution - finding nearest latitude-longitude 

from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))

tempDataList = [{'lat': 39.7612992, 'lon': -86.1519681}, 
                {'lat': 39.762241,  'lon': -86.158436 }, 
                {'lat': 39.7622292, 'lon': -86.1578917}]

v = {'lat': 39.7622290, 'lon': -86.1519750}
print(closest(tempDataList, v))

{'lat': 39.7612992, 'lon': -86.1519681}


In [6]:
# adding point column to df
df['point'] = [(x, y) for x,y in zip(df['lat'], df['lon'])]

In [7]:
df.tail(7)

,Store,lon,lat,point
293,2565,-2.892610,52.8970,"(52.897, -2.89261)"
294,1230,-3.866800,57.5185,"(57.5185, -3.8668)"
295,1089,-0.287350,51.5607,"(51.5607, -0.28735)"
296,538,0.861843,51.8544,"(51.8544, 0.861843)"
297,973,-4.254070,55.8459,"(55.8459, -4.25407)"
298,1066,-0.617720,51.1864,"(51.1864, -0.6177199999999999)"
299,2203,-0.133300,51.1167,"(51.1167, -0.1333)"


In [8]:
from scipy.spatial.distance import cdist

In [9]:
def closest_point(point, points):
    """ Find closest point from a list of points. """
    return points[cdist([point], points).argmin()]

def match_value(df, col1, x, col2):
    """ Match value x from col1 row to value in col2. """
    return df[df[col1] == x][col2].values[0]


In [10]:
tl = list(df['point'])
tl[:5]

[(51.2797, -0.836673),
 (55.8214, -4.08744),
 (51.3476, -0.799592),
 (51.4664, -2.6021799999999997),
 (53.5689, -2.42275)]

In [11]:
# using knn

from sklearn.neighbors import NearestNeighbors

def distance(p1, p2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    lon1, lat1 = p1
    lon2, lat2 = p2
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

points = tl

nbrs = NearestNeighbors(n_neighbors=2, metric=distance).fit(points)

distances, indices = nbrs.kneighbors(points)

result = distances[:, 1]

In [12]:
len(result)

300

In [13]:
result

array([  8.59656755,   6.1049544 ,   8.59656755,   2.43362356,
         4.47198077,   5.48712867,   2.89998426,  17.52605129,
         2.6684596 ,   7.93533518,   2.31342682,  20.93197162,
         3.27043196,   1.45798444,  17.52605129,  50.24568219,
        31.66771248,  10.78946304,   1.48620414,  26.83390248,
        39.83600758,   7.44481885,  22.35199435,  15.33237904,
        11.07917243,  14.76446136,  13.28009768,  14.63925675,
        21.28456305,   7.77873533,   4.82121392,  44.53777362,
        22.36417796,  43.02586313,   4.50080855,  18.70944083,
         4.64084257,  43.3661976 ,   4.01026994,  36.09520344,
         3.62270254,  24.73676751,   1.98076628,   7.46031905,
         1.48620414,   3.40594056,  69.34099116,  42.18862094,
        12.88940191,  40.07025531,  30.09809845,  29.40559247,
         5.14404015,  10.78946304,   9.88272321,  11.26323347,
        45.54666684,  44.77682794,   4.04616361,  17.56663102,
         5.14404015,  11.55644564,  10.14200074,  21.96

In [14]:
tl = list(df['point'].values)
tl[:5]

tlx = []
for x in tl:
    tlx.append([x[0],x[1]])

tlx[:5]

[[51.2797, -0.836673],
 [55.8214, -4.08744],
 [51.3476, -0.799592],
 [51.4664, -2.6021799999999997],
 [53.5689, -2.42275]]

In [15]:
# simple approach

# closest node
def closest_node(node, nodes):
    closest_index = cdist([node], nodes).argmin()
    return nodes[closest_index]

closest_node(tlx[0], tlx[1:])

[51.3476, -0.799592]

In [16]:
len(tlx)

300

In [17]:
fl = []
tlx__ = tlx

for i,x in enumerate(tlx):
    tlx__.remove(x)
    fl.append(closest_node(x, tlx__))
    tlx__.append(x)

In [18]:
len(fl)

300

In [19]:
df['closest'] = fl

In [20]:
df.sample(5)

,Store,lon,lat,point,closest
17,150,-2.431480,53.7869,"(53.7869, -2.43148)","[52.0192, -0.301697]"
178,1165,-0.290115,53.7583,"(53.7583, -0.290115)","[50.9837, 0.09239299999999999]"
135,1559,-1.602300,53.8590,"(53.858999999999995, -1.6023)","[52.0642, -0.7821279999999999]"
138,1534,-4.263900,53.2927,"(53.2927, -4.2639)","[57.1434, -2.1995299999999998]"
22,1492,-3.803080,53.2966,"(53.2966, -3.8030800000000005)","[51.5106, -0.33541]"


In [21]:
# adding point column to df
df['point'] = [(x, y) for x,y in zip(df['lat'], df['lon'])]

In [22]:
from scipy.spatial.distance import cdist

In [23]:
# closest node
def closest_node(node, nodes):
    closest_index = cdist([node], nodes).argmin()
    return nodes[closest_index]

In [24]:
tl = list(df['point'].values)

tlx = []
tlx__ = []
for x in tl:
    tlx.append([x[0],x[1]])
    tlx__.append([x[0],x[1]])

tlx[:5]

[[51.2797, -0.836673],
 [55.8214, -4.08744],
 [51.3476, -0.799592],
 [51.4664, -2.6021799999999997],
 [53.5689, -2.42275]]

In [25]:
fl = []

for i,x in enumerate(tlx):
    tlx__.remove(x)
    fl.append(closest_node(x, tlx__))
    tlx__.append(x)

In [26]:
df['closest'] = fl

In [27]:
df

,Store,lon,lat,point,closest
0,1063,-0.836673,51.2797,"(51.2797, -0.836673)","[51.3476, -0.799592]"
1,989,-4.087440,55.8214,"(55.8214, -4.08744)","[55.8688, -4.115419999999999]"
2,1061,-0.799592,51.3476,"(51.3476, -0.799592)","[51.2797, -0.836673]"
3,302,-2.602180,51.4664,"(51.4664, -2.6021799999999997)","[51.4591, -2.58153]"
4,212,-2.422750,53.5689,"(53.5689, -2.42275)","[53.5937, -2.3910400000000003]"
5,469,-3.011910,53.3970,"(53.397, -3.01191)","[53.4386, -2.98522]"
6,132,-2.640290,50.9610,"(50.961000000000006, -2.6402900000000002)","[50.9676, -2.61504]"
7,288,-2.414050,51.5332,"(51.5332, -2.41405)","[51.6878, -2.44592]"
8,761,-2.085180,52.4923,"(52.4923, -2.08518)","[52.4744, -2.0691599999999997]"
9,2126,-0.924921,50.8473,"(50.8473, -0.9249209999999999)","[50.803999999999995, -0.981709]"


In [28]:
l1 = [1,2,3,4,5]

# shallow copy
l2 = l1

for i,x in enumerate(l1):
    l2.remove(x)
    print(x, l1, l2)
    l2.append(x)

l1, l2

1 [2, 3, 4, 5] [2, 3, 4, 5]
3 [2, 4, 5, 1] [2, 4, 5, 1]
5 [2, 4, 1, 3] [2, 4, 1, 3]
3 [2, 4, 1, 5] [2, 4, 1, 5]
3 [2, 4, 1, 5] [2, 4, 1, 5]


([2, 4, 1, 5, 3], [2, 4, 1, 5, 3])

In [31]:
import copy

l1 = [1,2,3,4,5]

# deep copy
l2 = copy.deepcopy(l1)

In [32]:
for i,x in enumerate(l1):
    l2.remove(x)
    print(x, l1, l2)
    l2.append(x)

l1, l2

1 [1, 2, 3, 4, 5] [2, 3, 4, 5]
2 [1, 2, 3, 4, 5] [3, 4, 5, 1]
3 [1, 2, 3, 4, 5] [4, 5, 1, 2]
4 [1, 2, 3, 4, 5] [5, 1, 2, 3]
5 [1, 2, 3, 4, 5] [1, 2, 3, 4]


([1, 2, 3, 4, 5], [1, 2, 3, 4, 5])